# Lending Club Case Study - Exploratory Data Analysis

### Group Members 
 - Ruchir Mayank
 - Srivishnu Ayyagari



### Business Understanding:
- Lending Club(LC) is a platform where borrower's can borrow money at lower interest rate.On the other hand LC allows investor to get better return based on the interest they choose to lend to the borrowers.
- LC will make a loss when one or more borrowers start defaulting on the money to be repaid
- In some cases LC is able to recover some amount via settlement but the amount that is not recovered will be loss to the company

### Business Objectives
- LC wants to analyse the factors that can help them decide if the borrowers are likely to default and hence take the decision based on such data.


### Exploratory Data Analysis

#### Data Sourcing and cleaning

**Import required libraries**

In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='whitegrid')
import re

**Reading the loan data**

In [152]:
loan=pd.read_csv('C:/Users/ruchirm/Documents/UpGrad/Assignments/Lending club/loan.csv',encoding='ISO-8859-1',low_memory=False)
#have to set low-memory as false as the col 47 have mixed data type and read_csv is not able to process it without this option
loan.shape
loan.describe

<bound method NDFrame.describe of             id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0      1077501    1296599       5000         5000           4975.0   
1      1077430    1314167       2500         2500           2500.0   
2      1077175    1313524       2400         2400           2400.0   
3      1076863    1277178      10000        10000          10000.0   
4      1075358    1311748       3000         3000           3000.0   
...        ...        ...        ...          ...              ...   
39712    92187      92174       2500         2500           1075.0   
39713    90665      90607       8500         8500            875.0   
39714    90395      90390       5000         5000           1325.0   
39715    90376      89243       5000         5000            650.0   
39716    87023      86999       7500         7500            800.0   

             term int_rate  installment grade sub_grade  ...  \
0       36 months   10.65%       162.87     B        B2  ... 

**Data Cleaning**

We will first try to find out how many columns have missing or null values and what percentageof the values are missing in respective columns. Depending on the outcome we will decide if we want to fill the missing values or we want to remove them from our analysis.

In [153]:
nullValuePercentagePerColumn=round(loan.isnull().sum()/len(loan.id),4)*100
#nullValuePercentagePerColumn.loc[nullValuePercentagePerColumn>0]
nullValuePercentagePerColumn.loc[nullValuePercentagePerColumn>10]

desc                               32.58
mths_since_last_delinq             64.66
mths_since_last_record             92.99
next_pymnt_d                       97.13
mths_since_last_major_derog       100.00
annual_inc_joint                  100.00
dti_joint                         100.00
verification_status_joint         100.00
tot_coll_amt                      100.00
tot_cur_bal                       100.00
open_acc_6m                       100.00
open_il_6m                        100.00
open_il_12m                       100.00
open_il_24m                       100.00
mths_since_rcnt_il                100.00
total_bal_il                      100.00
il_util                           100.00
open_rv_12m                       100.00
open_rv_24m                       100.00
max_bal_bc                        100.00
all_util                          100.00
total_rev_hi_lim                  100.00
inq_fi                            100.00
total_cu_tl                       100.00
inq_last_12m    

We can see from above output that there are 54 column where 100% of the data is missing and there are additional 4 columns where more than 30% of the data is missing. We will drop these columns from our analysis

In [154]:
columnWithNullvalue=list(nullValuePercentagePerColumn[nullValuePercentagePerColumn>=30].index)
#len(columnWithNullvalue)
loan.drop(columnWithNullvalue,axis=1,inplace=True)
loan.shape

(39717, 53)

We will also look into the column where the unique value is same, in that case that copluimn does not adds any value to the analysis

In [155]:
uniqueValuePerColumn=loan.nunique()
uniqueValuePerColumn=uniqueValuePerColumn[uniqueValuePerColumn.values == 1]
uniqueValuePerColumn

pymnt_plan                    1
initial_list_status           1
collections_12_mths_ex_med    1
policy_code                   1
application_type              1
acc_now_delinq                1
chargeoff_within_12_mths      1
delinq_amnt                   1
tax_liens                     1
dtype: int64

Let's drop the above columns

In [156]:
loan.drop(labels = list(uniqueValuePerColumn.index),axis=1,inplace=True)
loan.shape

(39717, 44)

44 columns are remianing, we will examine remaining column for null values

In [157]:
nullValuePercentagePerColumn=round(loan.isnull().sum()/len(loan.id),4)*100
nullValuePercentagePerColumn[nullValuePercentagePerColumn>0]

emp_title               6.19
emp_length              2.71
title                   0.03
revol_util              0.13
last_pymnt_d            0.18
last_credit_pull_d      0.01
pub_rec_bankruptcies    1.75
dtype: float64

- We will analyse the unique value for each of the aformentioned columns with missing or null values
- From the data dictionary file we understand that the **emp_title** and **title** are free string that teh borrower provide. We will make an assumption that it will be hard to make a categorical decision based on these and hence we will drop the rows with null/missing values for these columns.
- Since the percentage of null/missing valuses for these coulmns are very less , I will still like to keep the column for future refrence
- **emp_length** column provides information about how long the borrower has beenn employeed. This information may be usefull but there is no additional information provided based on which we can fill teh missing value. So we will drop the rows with missing value for this also

In [158]:
loan=loan[~loan.emp_title.isnull()]
loan=loan[~loan.title.isnull()]
loan=loan[~loan.emp_length.isnull()]
loan.shape

(37193, 44)

Let's look into the unique values for columns which are not free strings

In [159]:
loan.emp_length.unique()

array(['< 1 year', '10+ years', '1 year', '3 years', '8 years', '9 years',
       '5 years', '4 years', '6 years', '2 years', '7 years'],
      dtype=object)

In [160]:
loan.revol_util.unique()

array(['9.40%', '21%', '53.90%', ..., '49.63%', '0.04%', '7.28%'],
      dtype=object)

In [161]:
loan.last_pymnt_d.unique()

array(['Apr-13', 'Jan-15', 'May-16', 'Nov-12', 'Jun-13', 'Sep-13',
       'Jul-12', 'Oct-13', 'May-13', 'Feb-15', 'Aug-15', 'Oct-12',
       'Sep-12', nan, 'Dec-12', 'Dec-14', 'Aug-13', 'Nov-13', 'Jan-14',
       'Apr-14', 'Aug-14', 'Oct-14', 'Aug-12', 'Jul-14', 'Jul-13',
       'Jan-16', 'Apr-15', 'Feb-14', 'Sep-14', 'Jun-12', 'Feb-13',
       'Mar-13', 'May-14', 'Mar-15', 'Jan-13', 'Dec-13', 'Jun-14',
       'Feb-12', 'Mar-14', 'Apr-12', 'Sep-15', 'Nov-15', 'Mar-16',
       'Feb-16', 'Jan-12', 'Oct-15', 'Nov-14', 'Mar-12', 'May-12',
       'Apr-16', 'Dec-15', 'Jun-15', 'May-15', 'Jul-15', 'Dec-11',
       'Nov-11', 'Oct-11', 'Sep-11', 'Aug-11', 'Jul-11', 'Jun-11',
       'May-11', 'Apr-11', 'Mar-11', 'Feb-11', 'Jan-11', 'Dec-10',
       'Nov-10', 'Oct-10', 'Sep-10', 'Aug-10', 'Jul-10', 'Jun-10',
       'May-10', 'Apr-10', 'Mar-10', 'Feb-10', 'Jan-10', 'Dec-09',
       'Nov-09', 'Oct-09', 'Sep-09', 'Aug-09', 'Jul-09', 'Jun-09',
       'May-09', 'Apr-09', 'Mar-09', 'Feb-09', 'Jan-09', 

In [162]:
loan.last_credit_pull_d.unique()

array(['Sep-13', 'Apr-16', 'May-16', 'Jan-16', 'Dec-14', 'Mar-13',
       'Dec-15', 'Aug-13', 'Nov-12', 'Mar-14', 'Apr-15', 'May-14',
       'Jul-15', 'Feb-16', 'Mar-16', 'Sep-12', 'May-13', 'Jan-15',
       'Jun-12', 'Mar-15', 'Dec-12', 'Sep-14', 'Feb-14', 'Jun-15',
       'Oct-13', 'Apr-14', 'Oct-14', 'Feb-13', 'Nov-15', 'Jul-14',
       'Sep-15', 'Oct-12', 'Nov-13', 'Nov-14', 'Feb-12', 'Oct-15',
       'Apr-12', 'Aug-15', 'Jun-14', 'Aug-12', 'Jan-12', 'Aug-14',
       'Jun-13', 'Dec-13', 'May-12', 'Jul-12', 'Jan-14', 'Jul-13',
       'Apr-13', 'May-15', 'Feb-15', 'Mar-12', 'Nov-11', 'Dec-11',
       'Oct-11', 'Sep-11', 'Jan-13', 'Aug-11', 'Jul-11', 'Jun-11',
       'May-11', 'Apr-11', 'Mar-11', 'Feb-11', 'Jan-11', 'Dec-10',
       'Nov-10', 'Oct-10', 'Sep-10', 'Aug-10', 'Jul-10', 'Jun-10',
       'May-10', 'Apr-10', 'Feb-10', 'Mar-10', 'Jan-10', 'Dec-09',
       'Nov-09', 'Oct-09', 'Sep-09', 'Jul-09', 'Aug-09', 'May-09',
       'Jun-09', 'Apr-09', 'Mar-09', 'Feb-09', 'Jan-09', 'Dec-

In [163]:
loan.pub_rec_bankruptcies.unique()   

array([ 0.,  1.,  2., nan])

Let's remove  the rows with null values in the above columns


In [164]:
loan=loan[~loan.revol_util.isnull()]
loan=loan[~loan.last_pymnt_d.isnull()]
loan=loan[~loan.last_credit_pull_d.isnull()]
loan=loan[~loan.pub_rec_bankruptcies.isnull()]
loan.shape

(36433, 44)

At this point I will fall back to some manual interpetation by looking into the data dictonary file and checking if any of  the remaining columns are required.

In [165]:
loan.nunique().sort_values()

term                           2
pub_rec_bankruptcies           3
loan_status                    3
verification_status            3
home_ownership                 4
pub_rec                        5
grade                          7
inq_last_6mths                 9
delinq_2yrs                   11
emp_length                    11
purpose                       14
sub_grade                     35
open_acc                      38
addr_state                    49
issue_d                       52
total_acc                     79
last_credit_pull_d            96
last_pymnt_d                  97
int_rate                     334
earliest_cr_line             510
zip_code                     806
loan_amnt                    863
funded_amnt                 1024
out_prncp                   1064
out_prncp_inv               1065
revol_util                  1082
total_rec_late_fee          1194
collection_recovery_fee     2386
dti                         2857
recoveries                  3585
annual_inc

**id**:A unique LC assigned ID for the loan listing.<br>
**member_id**: A unique LC assigned Id for the borrower member.<br>
**url**:A unique LC assigned Id for the borrower member.<br>
    
The above values are just additional attribute which does not have any corelation with the borrower or lender patterns. So these should be safe to remove.

<br>**funded_amnt**: The value is almost same as the loan amount
<br>**funded_amt_inv**: The value is almost same as the loan amount

These values  may cause a small varitaionin anakysis but since for majority of the data value remains the same these will have negligible impact and hence safe to drop.
     
   

In [166]:
dropColumns=['id','member_id','url','funded_amnt','funded_amnt_inv']
loan.drop(dropColumns,axis=1,inplace=True)
loan.shape

(36433, 39)

From the data dict file we saw that emp length can take values from 0 to 10 but from previous analysis we see that thecolumn has mix representation in values. So we will stndarise teh values from 0 to 10

In [167]:
empLengthDef = {
    '< 1 year' : 0,
    '1 year' : 1,
    '2 years' : 2,
    '3 years' : 3,
    '4 years' : 4,
    '5 years' : 5,
    '6 years' : 6,
    '7 years' : 7,
    '8 years' : 8,
    '9 years' : 9,
    '10+ years' : 10
}
loan.replace({'emp_length':empLengthDef},inplace=True)

Till now we have reduced the dataset to 399 columns. But the data in each of the columns are not in a standard format.
Similar to what we have doen for emp_length we will have to analyse the columns and stadarise the values

In [168]:
loan.total_pymnt.value_counts().head()

11196.569430    26
11784.232230    16
13148.137860    15
10956.775960    13
5478.387981     13
Name: total_pymnt, dtype: int64

In [134]:
loan.total_rec_late_fee.value_counts().head()

0.0     34618
15.0      224
30.0       51
15.0       49
15.0       42
Name: total_rec_late_fee, dtype: int64

In [169]:
loan.collection_recovery_fee.value_counts().head()

0.00    33100
1.20       10
2.00        9
2.02        8
1.69        8
Name: collection_recovery_fee, dtype: int64

We will keep the precsion to 2 decimial poinyts for above columns

In [170]:
loan.total_pymnt=round(loan.total_pymnt,2)
loan.total_rec_late_fee= round(loan.total_rec_late_fee,2)
loan.collection_recovery_fee= round(loan.collection_recovery_fee,2)
#loan.collection_recovery_fee.value_counts().head()

Similarly we will standarise any date time column to standard dattime format of **mon-year**<br>
Refernce to do this : https://stackabuse.com/how-to-format-dates-in-python/

In [142]:
loan.issue_d = pd.to_datetime(loan.issue_d, format='%b-%y')
loan.earliest_cr_line = pd.to_datetime(loan.earliest_cr_line, format='%b-%y')
loan.last_pymnt_d = pd.to_datetime(loan.last_pymnt_d, format='%b-%y')
loan.last_credit_pull_d = pd.to_datetime(loan.last_credit_pull_d, format='%b-%y')

Looking into the loan.csv file and checking teh columns we see few more anamolies in data that needs to be fixed.
Doing  those belows along with inline comments for easier reference

In [175]:
loan.term.value_counts()#removingthe "month" string from column
#loan['term']=loan['term'].replace("month",'',regex=True)
# The replace function is somehow adding s at the end. Hence I am using brute force to split based on the pattern
loan['term'] = loan.term.apply(lambda x: x.split()[0])
loan.term.value_counts()

36    26377
60    10056
Name: term, dtype: int64

This seems to be the last of cleaning effort. Removing % from rate columns. If anyting is left we will update when we encounter the occurrence

In [177]:
loan.int_rate.head()

1    15.27%
3    13.49%
4    12.69%
5     7.90%
6    15.96%
Name: int_rate, dtype: object

In [178]:
loan.revol_util.head()

1     9.40%
3       21%
4    53.90%
5    28.30%
6    85.60%
Name: revol_util, dtype: object

In [181]:
loan['int_rate'] = loan.int_rate.apply(lambda x: x.split('%')[0])
loan['revol_util']=loan.revol_util.apply(lambda x: x.split('%')[0])
#loan.revol_util.head()

### Derived Metrics
We have cleaned up the data now we should derive certain columns that will help us in doing the analysis